In [2]:
import numpy as np
from scipy.linalg import lstsq
from scipy.linalg import null_space
from scipy.integrate import quad


In [4]:
'''# 定义p(r)
def p_r(r, c):
    p_r = c[0]+sum(c[i] * r**i for i in range(2, len(c)))
    return p_r

def dp_r(r, c):
    dp_r = sum(i * c[i] * r**(i-1) for i in range(2, len(c)))
    return dp_r

def ddp_r(r, c):
    ddp_r = sum(i * (i-1) * c[i] * r**(i-2) for i in range(2, len(c)))
    return ddp_r'''

'# 定义p(r)\ndef p_r(r, c):\n    p_r = c[0]+sum(c[i] * r**i for i in range(2, len(c)))\n    return p_r\n\ndef dp_r(r, c):\n    dp_r = sum(i * c[i] * r**(i-1) for i in range(2, len(c)))\n    return dp_r\n\ndef ddp_r(r, c):\n    ddp_r = sum(i * (i-1) * c[i] * r**(i-2) for i in range(2, len(c)))\n    return ddp_r'

In [11]:
#小写p(r)系列构建关于c0, c2, c3, c4的线性方程组
#range(i, j) produces i, i+1, i+2, ..., j-1. start defaults to 0
def p_r(r):
    A=np.zeros(4)
    A[0]=1
    for i in range(1,len(A)):
        A[i]=r**(i+1)
    return A

def dp_r(r):
    dA=np.zeros(4)
    for i in range(1,len(A)):
        dA[i]=(i+1)*r**(i)
    return dA

def ddp_r(r):
    ddA=np.zeros(4)
    for i in range(1,len(A)):
        ddA[i]=i*(i+1)*r**(i-1)
    return ddA
# A[0]~c_0, A[1]~c_2, A[2]~c_3, A[3]~c_4

'''p_3p=np.array([p_r(2.14),dp_r(2.14),ddp_r(2.14)])
print(p_3p)
print(null_space(p_3p))'''

#l_vec(r)写出关于线性方程组的矩阵
def l_vec(r):
    return np.array([p_r(r), dp_r(r), ddp_r(r)])

print(l_vec(2))
print(null_space(l_vec(2.14)))
A=np.zeros(4)
print(len(A))

[[ 1.  4.  8. 16.]
 [ 0.  4. 12. 32.]
 [ 0.  2. 12. 48.]]
[[-0.54206174]
 [ 0.71018657]
 [-0.44248385]
 [ 0.07753806]]
4


In [18]:
#大写P是rR_AE的代指，此处已经经过在截断半径处的局部三次多项式拟合
def P(r, coeff):
    return sum(coeff[i] * r**i for i in range(0, len(coeff)))

def dP(r, coeff):
    return sum(i * coeff[i] * r**(i-1) for i in range(0, len(coeff)))

def ddP(r, coeff):  
    return sum(i * (i-1) * coeff[i] * r**(i-2) for i in range(0, len(coeff)))

'''de0_3p=np.log(abs(P(2.14,coeff_3p)/2.14**2))
de1_3p=dP(2.14,coeff_3p)/P(2.14,coeff_3p)-2/2.14
de2_3p=ddP(2.14,coeff_3p)/P(2.14,coeff_3p)-\
(dP(2.14,coeff_3p)/P(2.14,coeff_3p))**2+2/2.14**2
print(de0_3p)'''

def r_vec(r,coeff,l):
    de0=np.log(abs(P(r,coeff)/r**(l+1)))
    de1=dP(r,coeff)/P(r,coeff)-(l+1)/r
    de2=ddP(r,coeff)/P(r,coeff)-\
    (dP(r,coeff)/P(r,coeff))**2+(l+1)/r**2
    return np.array([de0,de1,de2])

coeff=np.array([1,1,1,1])
print(P(2,coeff))

15


In [31]:
coeff_3p=np.array([1.04066,-1.95036,0.73568,-0.08737])
r_vec_3p=r_vec(2.14,coeff_3p,1)
l_vec_3p=l_vec(2.14)
print(r_vec_3p)
print(l_vec_3p)

x, residuals, rank, s = lstsq(l_vec_3p,r_vec_3p)

print("广义解:", x)
print("矩阵的秩:", rank)
print("奇异值:", s)

[-1.99925024 -0.93134112 -0.126824  ]
[[ 1.          4.5796      9.800344   20.97273616]
 [ 0.          4.28       13.7388     39.201376  ]
 [ 0.          2.         12.84       54.9552    ]]
广义解: [-0.69651895 -0.46457754  0.10584358 -0.01013008]
矩阵的秩: 3
奇异值: [73.81423413  6.4451246   0.74489052]


In [28]:
def I_left(x,null_space,c,l,r_c):
    def integrand(r):
        exponent=x[0]+null_space[0]*c+(x[1]+null_space[1]*c)*r**2+\
        (x[2]+null_space[2]*c)*r**3+(x[3]+null_space[3]*c)*r**4
        return r**(2*l+2)*np.exp(exponent)
    return quad(integrand,0,r_c)[0]

# 定义误差函数
def F(I_right, x, null_space, c, l, r_c):
    return I_left(x,null_space,c,l,r_c) - I_right

print(F(0.3,x,null_space(l_vec(2.14)),100,1,2.14))

0.03138588716997054


In [35]:
def false_position_method(I_right, c_low, c_high, l, r_c, x, null_space, tol=1e-4, max_iter=500):
    i=0
    for i in range(max_iter):
        # 计算两端误差值
        f_low = F(I_right, x, null_space, c_low, l, r_c)
        f_high = F(I_right, x, null_space, c_high, l, r_c)

        # 更新新点
        c_new = c_low - f_low * (c_high - c_low) / (f_high - f_low)
        f_new = F(I_right, x, null_space, c_new, l, r_c)

        # 检查收敛
        if abs(f_new) < tol:
            return c_new

        # 更新区间
        if f_new * f_low < 0:
            c_high = c_new
            i = i+1
        else:
            c_low = c_new
            i = i+1

    raise ValueError("未能收敛")

In [36]:
I_r_3p=0.337544969
c_3p=false_position_method(I_r_3p, -0.5, 0.5, 1, 2.14, x, null_space(l_vec(2.14)))
print(c_3p)

1633.2478522195697


In [17]:
def cal(x, null_space,c):
    for i in range(len(x)):
        x[i]=x[i]+null_space[i]*c
    return x

x1=cal(x,null_space(l_vec(2.14)),c_3p)
print(x1)

[-886.01768836 1159.44611693 -722.57994737  126.62873516]


C:\Users\msy_b\AppData\Local\Temp\ipykernel_20012\3256658590.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i]=x[i]+null_space[i]*c
